In [1]:
import sys
from pathlib import Path

# Add project root (parent of notebooks/) to sys.path
project_root = Path.cwd().parent
sys.path.insert(0, str(project_root))
print("Project root:", project_root)

Project root: D:\UMNG-2025\FakeVoice\FakeVoice


In [3]:
from fakevoicefinder import ExperimentConfig, CreateExperiment, ModelLoader, ConfigError


In [5]:
# --- Configure your experiment ---
cfg = ExperimentConfig()

# REQUIRED: name of the experiment folder under outputs/
cfg.run_name       = "exp_zip_only_v3"   # <<< IMPORTANT: this avoids the exp_name error

# Data: folder must contain the two zips: reals.zip and fakes.zip
cfg.data_path      = "../dataset"
cfg.reals_zip      = "reals.zip"      # change to your actual file names if different
cfg.fakes_zip      = "fakes.zip"

# Optional user models path
cfg.models_path    = "../models"

# Benchmark models (can be an empty list)
cfg.models_list    = ["AlexNet", "ResNet18"]

# Train type: 'scratch' | 'pretrain' | 'both'
cfg.type_train     = "both"

# Transforms to generate during data preparation
cfg.transform_list = ["mel", "log"]

# Training defaults (kept in manifest for traceability)
cfg.epochs         = 20
cfg.batch_size     = 32
cfg.optimizer      = "Adam"
cfg.learning_rate  = 0.003
cfg.patience       = 10
cfg.device         = "cpu"   # or "gpu"
cfg.seed           = 23
cfg.num_workers    = 4

# Used by ModelLoader: number of input channels (1 for mel/log, 3 for RGB)
cfg.input_channels = 1

print(cfg.summary())

ExperimentConfig:
  batch_size     : 32
  cache_features : True
  data_path      : ../dataset
  device         : cpu
  epochs         : 20
  eval_metric    : ['accuracy', 'F1']
  fakes_zip      : fakes.zip
  flag_train     : True
  input_channels : 1
  learning_rate  : 0.003
  models_list    : ['AlexNet', 'ResNet18']
  models_path    : ../models
  num_workers    : 4
  optimizer      : Adam
  outputs_path   : outputs
  patience       : 10
  reals_zip      : reals.zip
  run_name       : exp_zip_only_v3
  save_best_only : True
  save_models    : True
  seed           : 23
  transform_list : ['mel', 'log']
  type_train     : both


In [7]:
try:
    cfg.validate()
    print("Config validation ✅")
except ConfigError as e:
    print("Config validation error:", e)
    raise

Config validation ✅


In [9]:
exp = CreateExperiment(cfg)  # uses cfg.run_name internally
exp_dict = exp.build(make_dirs=True)
print("Experiment root:", exp.root)
print("Manifest path :", exp.root / "experiment.json")


Experiment root: D:\UMNG-2025\FakeVoice\FakeVoice\outputs\exp_zip_only_v3
Manifest path : D:\UMNG-2025\FakeVoice\FakeVoice\outputs\exp_zip_only_v3\experiment.json


In [11]:
# One-liner: load -> split -> save originals -> transforms -> update manifest
TRAIN_RATIO = 0.8
summary = exp.prepare_data(train_ratio=TRAIN_RATIO, seed=cfg.seed)
summary


mel • train:   0%|          | 0/656 [00:00<?, ?file/s]

C:\Users\Gpach\anaconda3\Lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated and will be removed in a future release
  "class": algorithms.Blowfish,


mel • test:   0%|          | 0/183 [00:00<?, ?file/s]

log • train:   0%|          | 0/656 [00:00<?, ?file/s]

log • test:   0%|          | 0/183 [00:00<?, ?file/s]

{'load': {'reals': 824, 'fakes': 107},
 'split': {'train': {'total': 744, 'reals': 658, 'fakes': 86},
  'test': {'total': 187, 'reals': 166, 'fakes': 21}},
 'save_original': {'train': 744, 'test': 187},
 'transforms': {'mel': {'train': 656, 'test': 183},
  'log': {'train': 656, 'test': 183}}}

In [13]:
import json
from pprint import pprint

with open(exp.root / "experiment.json", "r", encoding="utf-8") as f:
    manifest = json.load(f)

pprint({
    "experiment_root": manifest["_experiment_root"],
    "train_original_path": manifest["experiment"]["train_data"]["original_dataset"]["path"],
    "train_num_items": manifest["experiment"]["train_data"]["original_dataset"]["num_items"],
    "test_original_path": manifest["experiment"]["test_data"]["original_dataset"]["path"],
    "test_num_items": manifest["experiment"]["test_data"]["original_dataset"]["num_items"],
    "transforms_train": {k: v["path"] for k, v in manifest["experiment"]["train_data"]["transforms_dataset"].items()},
    "transforms_test":  {k: v["path"] for k, v in manifest["experiment"]["test_data"]["transforms_dataset"].items()},
})


{'experiment_root': 'outputs/exp_zip_only_v3',
 'test_num_items': 183,
 'test_original_path': 'outputs/exp_zip_only_v3/datasets/test/original',
 'train_num_items': 656,
 'train_original_path': 'outputs/exp_zip_only_v3/datasets/train/original',
 'transforms_test': {'log': 'outputs/exp_zip_only_v3/datasets/test/transforms/log',
                     'mel': 'outputs/exp_zip_only_v3/datasets/test/transforms/mel'},
 'transforms_train': {'log': 'outputs/exp_zip_only_v3/datasets/train/transforms/log',
                      'mel': 'outputs/exp_zip_only_v3/datasets/train/transforms/mel'}}


In [15]:
from pprint import pprint
loader = ModelLoader(exp)

# Benchmarks according to cfg.models_list and cfg.type_train
bench = loader.prepare_benchmarks(add_softmax=True, input_channels=cfg.input_channels)
print("Benchmark models saved:")
pprint(bench)

# User models (if any .pt/.pth under cfg.models_path)
user = loader.prepare_user_models(add_softmax=True, input_channels=cfg.input_channels)
print("User models saved:")
pprint(user)


Benchmark models saved:
{'AlexNet': {'pretrain': 'outputs/exp_zip_only_v3/models/loaded/AlexNet_pretrain.pt',
             'scratch': 'outputs/exp_zip_only_v3/models/loaded/AlexNet_scratch.pt'},
 'ResNet18': {'pretrain': 'outputs/exp_zip_only_v3/models/loaded/ResNet18_pretrain.pt',
              'scratch': 'outputs/exp_zip_only_v3/models/loaded/ResNet18_scratch.pt'}}
User models saved:
{'SimpleCNN_scripted.pt': 'outputs/exp_zip_only_v3/models/loaded/SimpleCNN_scripted_usermodel_jit.pt'}


In [17]:
def print_tree(root: Path, max_depth: int = 3, prefix: str = ""):
    if max_depth < 0:
        return
    try:
        entries = sorted(root.iterdir(), key=lambda p: (p.is_file(), p.name.lower()))
    except FileNotFoundError:
        return
    for e in entries:
        print(prefix + ("📄 " if e.is_file() else "📁 ") + e.name)
        if e.is_dir():
            print_tree(e, max_depth - 1, prefix + "   ")

print_tree(exp.root, max_depth=3)

📁 datasets
   📁 test
      📁 original
         📁 fakes
         📁 reals
      📁 transforms
         📁 log
         📁 mel
   📁 train
      📁 original
         📁 fakes
         📁 reals
      📁 transforms
         📁 log
         📁 mel
📁 models
   📁 loaded
      📄 AlexNet_pretrain.pt
      📄 AlexNet_scratch.pt
      📄 ResNet18_pretrain.pt
      📄 ResNet18_scratch.pt
      📄 SimpleCNN_scripted_usermodel_jit.pt
   📁 trained
📄 experiment.json


In [19]:
with open(exp.root / "experiment.json", "r", encoding="utf-8") as f:
    manifest = json.load(f)

models_section = manifest["experiment"].get("models", {})
print("Model keys:", list(models_section.keys()))
for name, entry in models_section.items():
    print("—", name)
    print("   loaded_variants :", entry.get("loaded_variants"))
    print("   trained_path    :", entry.get("trained_path"))

Model keys: ['AlexNet', 'ResNet18', 'usermodel_SimpleCNN_scripted.pt']
— AlexNet
   loaded_variants : {'scratch': 'outputs/exp_zip_only_v3/models/loaded/AlexNet_scratch.pt', 'pretrain': 'outputs/exp_zip_only_v3/models/loaded/AlexNet_pretrain.pt'}
   trained_path    : None
— ResNet18
   loaded_variants : {'scratch': 'outputs/exp_zip_only_v3/models/loaded/ResNet18_scratch.pt', 'pretrain': 'outputs/exp_zip_only_v3/models/loaded/ResNet18_pretrain.pt'}
   trained_path    : None
— usermodel_SimpleCNN_scripted.pt
   loaded_variants : {'usermodel_jit': 'outputs/exp_zip_only_v3/models/loaded/SimpleCNN_scripted_usermodel_jit.pt'}
   trained_path    : None
